In [ ]:
from variational_autoencoder import VariationalAutoEncoder
from verification_net import VerificationNet
from stacked_mnist import StackedMNIST, DataMode

import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("mps")
torch.mps.set_per_process_memory_fraction(0.)

model_file = Path("models/vae-basic")

In [ ]:
class StackedVAE(VariationalAutoEncoder):
    def __init__(self, latent_space_size: int = 64) -> None:
        super().__init__(latent_space_size)

    def forward(self, x):
        nb_channels = x.shape[1]
        mus, log_vars = [], []
        x_hat = []
        for channel in range(nb_channels):
            x_channel = x[:, [channel], :, :]
            mu, log_var = self.encoder(x_channel)

            mus.append(mu.view(-1, 1, self.latent_space_size))
            log_vars.append(log_var.view(-1, 1, self.latent_space_size))

            std = torch.sqrt(torch.exp(log_var))
            eps = torch.randn_like(std)

            z = mu + eps * std
            x_hat.append(self.decoder(z))

        mus = torch.cat(mus, dim=1)
        log_vars = torch.cat(log_vars, dim=1)
        x_hat = torch.cat(x_hat, dim=1)
        return (mus, log_vars), x_hat


In [ ]:
VAE = StackedVAE()
VAE.load_state_dict(torch.load(model_file))

VAE.to(device)

In [ ]:
def loss(X, params):
    x_hat, x = X
    mu, log_var = params
    
    BCE = F.binary_cross_entropy(x_hat, x, reduction='mean')
    KLD = torch.mean(- 0.5 * torch.mean(1 + log_var - mu.pow(2) - torch.exp(log_var), axis=1))

    return BCE + .06 * KLD

In [ ]:
mode = DataMode.COLOR | DataMode.BINARY

In [ ]:
trainset = StackedMNIST(train=True, mode=mode)
testset = StackedMNIST(train=False, mode=mode)

In [ ]:
batch_size = 1
data = DataLoader(trainset, shuffle=True, batch_size=batch_size)
x, _, labels = next(iter(data))

In [ ]:
train_loader = DataLoader(dataset=trainset, shuffle=True, batch_size=2048)
test_loader = DataLoader(dataset=testset, shuffle=True, batch_size=2048)

In [ ]:
from util import tile_tv_images, tile_pil_images
from torchvision.transforms import ToPILImage

def print_reconstructed_img(dataset, batch_size: int = 25):
    data = DataLoader(dataset, shuffle=True, batch_size=batch_size)
    imgs, _, labels = next(iter(data))

    _, imgs_pred = VAE(imgs.to(device))
    
    labels = labels.detach().numpy()
    imgs_pil = [ ToPILImage()(img_pred.cpu()) for img_pred in imgs_pred ]
    tile_pil_images(images=imgs_pil)

In [ ]:
data = DataLoader(trainset, shuffle=True, batch_size=1000)
imgs, _, labels = next(iter(data))

_, imgs_pred = VAE(imgs.to(device))
imgs_pred.shape

In [ ]:
print_reconstructed_img(trainset, batch_size=16)

In [ ]:
print_reconstructed_img(testset, batch_size=16)

In [ ]:
VerifNet = VerificationNet(force_learn=False, file_name='models/verification_model')

data = DataLoader(trainset, shuffle=True, batch_size=10000)
imgs, _, labels = next(iter(data))

labels = labels.detach().numpy()
_, preds = VAE(imgs.to(device))

cov = VerifNet.check_class_coverage(data=imgs, tolerance=.8)
pred, acc = VerifNet.check_predictability(data=preds, correct_labels=labels, tolerance=.8)
print(f"Coverage: {100*cov:.2f}%")
print(f"Predictability: {100*pred:.2f}%")
print(f"Accuracy: {100 * acc:.2f}%")

In [ ]:
data = DataLoader(testset, shuffle=True, batch_size=10000)
imgs, _, labels = next(iter(data))

labels = labels.detach().numpy()
_, preds = VAE(imgs.to(device))

cov = VerifNet.check_class_coverage(data=imgs, tolerance=.8)
pred, acc = VerifNet.check_predictability(data=preds, correct_labels=labels, tolerance=.8)
print(f"Coverage: {100*cov:.2f}%")
print(f"Predictability: {100*pred:.2f}%")
print(f"Accuracy: {100 * acc:.2f}%")

In [ ]:
def errorbar_classification(classes, mean_accuracy, std_accuracy):
    x = classes
    width = 0.35

    fig, ax = plt.subplots()
    rects1 = ax.bar(x, mean_accuracy, width, yerr=std_accuracy, label='Accuracy')

    ax.set_ylabel('p(x)')
    ax.set_title('p(x)')
    ax.set_xticks(x)
    ax.set_xticklabels(classes)
    ax.legend()

    plt.show()

In [ ]:
data = DataLoader(testset, shuffle=True, batch_size=10000)
imgs, _, labels = next(iter(data))

labels = labels.detach().numpy()
imgs = imgs.to(device)
_, preds = VAE(imgs)

In [ ]:
nb_samples = 100

In [ ]:
from tqdm import tqdm

class_accuracies = [[] for _ in range(1000)]
probas = []
nb_channels = 3

for x, label in tqdm(zip(imgs, labels)):
    x = x.view(1, 3, 28, 28)
    x_hat = []
    for channel in range(nb_channels):
        x_channel = x[:, [channel], :, :]
        
        mu, log_var = VAE.encoder(x_channel)
        std = torch.sqrt(torch.exp(log_var))
        eps = torch.randn_like(std.expand(nb_samples,64)).to(device)

        z = mu + eps * std
        x_hat.append(VAE.decoder(z))

    x_hat = torch.cat(x_hat, dim=1)
    

    x = x.expand(nb_samples, 3, 28, 28)
    class_accuracies[label].append((-F.binary_cross_entropy(x_hat, x)).exp().mean().cpu().detach().numpy())
    probas.append((-F.binary_cross_entropy(x_hat, x)).exp().mean().cpu().detach().numpy())
    
mean_accuracy = [np.mean(acc) for acc in class_accuracies]
std_accuracy = [np.std(acc) for acc in class_accuracies]

probas = np.array(probas)

In [ ]:
errorbar_classification(np.arange(1000), mean_accuracy=mean_accuracy, std_accuracy=std_accuracy)

In [ ]:
def top_anomalous(k, losses, preds, labels, is_bottom = True):
    idx = np.argsort(losses)[:k] if is_bottom else np.flip(np.argsort(losses))[:k]

    new_labels = [ 
        str(f"p(x={labels[_id]}) = {int(losses[_id] * 1e3) / 1e3}") for _id in idx
        ]
    
    tile_tv_images(images=preds[idx], labels=new_labels)

In [ ]:
top_anomalous(15, probas, preds.permute(0, 2, 3, 1).cpu().detach().numpy(), labels)

In [ ]:
top_anomalous(15, probas, imgs.permute(0, 2, 3, 1).cpu().detach().numpy(), labels)

In [ ]:
top_anomalous(15, probas, preds.permute(0, 2, 3, 1).cpu().detach().numpy(), labels, is_bottom=False)


In [ ]:
x_values = np.argsort(mean_accuracy)[:16]
is_color = preds.shape[1] == 3

new_labels = [ 
    str(f"p(x={x_value})={int(mean_accuracy[x_value] * 1e3) / 1e3}") for x_value in x_values
    ]
np_imgs_predicted = preds.reshape(-1, 28, 28).cpu().detach().numpy() if not is_color else preds.permute(0,2,3,1).cpu().detach().numpy()

anomalies_pred = np.array([ np_imgs_predicted[np.argwhere(labels == x_value)[:,0]][0] for x_value in x_values ])

tile_tv_images(images=anomalies_pred, labels=new_labels)
